In [1]:
import pandas as pd
import os
import glob

In [2]:
def process_capacity_files_to_dataframe(folder_path: str) -> pd.DataFrame:
    """
    读取指定文件夹内所有 'lat_lon_capacity.csv' 格式的文件，
    将它们合并成一个单一的 Pandas DataFrame。

    Args:
        folder_path (str): 存放 CSV 文件的文件夹路径。

    Returns:
        pd.DataFrame: 一个合并后的 DataFrame，列包括 'lon', 'lat', 
                      以及从文件中读取到的所有设备类型 (如 'WT', 'PV' 等)。
                      如果找不到文件或文件夹，则返回一个空的 DataFrame。
    """
    # 构造文件搜索模式，以确保只匹配到目标文件
    file_pattern = os.path.join(folder_path, '*_*_capacity.csv')
    all_files = glob.glob(file_pattern)

    # 如果没有找到任何文件，打印警告并返回一个空的DataFrame
    if not all_files:
        print(f"警告: 在 '{folder_path}' 路径下没有找到符合 '*_*_capacity.csv' 模式的文件。")
        return pd.DataFrame()

    # 用于存储每一行数据的列表
    data_list = []

    # 遍历所有找到的文件路径
    for file_path in all_files:
        try:
            # 从文件名解析 lat 和 lon
            filename = os.path.basename(file_path)
            parts = filename.replace('_capacity.csv', '').split('_')
            
            # 确保文件名至少有两部分可以解析
            if len(parts) >= 2:
                lat = float(parts[0])
                lon = float(parts[1])
            else:
                print(f"警告: 文件名 '{filename}' 格式不正确，无法解析经纬度。已跳过此文件。")
                continue

            # 读取CSV文件，并将“长”数据转换为“宽”数据（字典）
            temp_df = pd.read_csv(file_path)
            device_data = temp_df.set_index('Device')['Optimal_Capacity'].to_dict()

            # 将经纬度信息与设备数据合并
            row_data = {'lat': lat, 'lon': lon}
            row_data.update(device_data)

            # 将该行数据添加到列表中
            data_list.append(row_data)

        except Exception as e:
            # 捕获可能的错误，如文件读取失败、数据格式错误等
            print(f"处理文件 {file_path} 时发生错误: {e}。已跳过此文件。")
            continue
    
    # 如果列表为空（例如所有文件都处理失败），返回空DataFrame
    if not data_list:
        return pd.DataFrame()

    # 从数据列表创建最终的DataFrame
    final_df = pd.DataFrame(data_list)

    # 重新排列列的顺序，使 'lon' 和 'lat' 在最前面
    # 获取设备列的名称并排序，以保证每次运行列顺序一致
    device_columns = sorted([col for col in final_df.columns if col not in ['lat', 'lon']])
    desired_order = ['lon', 'lat'] + device_columns
    
    # 应用新的列顺序，并填充可能出现的缺失值（如果某些文件缺少某个设备）
    final_df = final_df[desired_order].fillna(0)

    return final_df

In [3]:
df_capacity_0 = process_capacity_files_to_dataframe('output_0')
df_capacity_2020 = process_capacity_files_to_dataframe('output_2020')
df_capacity_2050 = process_capacity_files_to_dataframe('output_2050')

In [4]:
df_capacity_2020

,lon,lat,AC,CES,CHP,EB,ESS,FC,H2S,LNG,LNGV,PEM,PV,TES,WEC,WT
0,103.626017,-0.030217,155.420147,772.317060,0.0,0.0,11.633788,26.864989,601.983762,0.0,0.0,11.583597,85.978159,0.0,100.000000,7.118000
1,103.659208,-0.035119,34.352596,150.453257,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,7.220369,0.0,30.945258,1.517734
2,130.124424,-0.045282,130.619851,687.451257,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,33.912247,0.0,111.180000,3.605595
3,98.276855,-0.061393,330.643071,1610.936086,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,110.502614,0.0,255.000000,3.764000
4,98.326246,-0.074667,330.182418,1614.027909,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,111.570440,0.0,255.000000,3.589000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893,126.054181,9.873549,306.987833,1593.163663,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,87.037088,0.0,255.000000,1.108000
1894,125.527599,9.884029,300.933453,1568.599821,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,81.050055,0.0,255.000000,0.916000
1895,125.586853,9.889078,299.393638,1580.246944,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,79.429865,0.0,255.000000,1.204333
1896,125.948905,9.912293,306.290702,1570.158663,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,86.507599,0.0,255.000000,0.903667


In [5]:
df_capacity_2050

,lon,lat,AC,CES,CHP,EB,ESS,FC,H2S,LNG,LNGV,PEM,PV,TES,WEC,WT
0,103.626017,-0.030217,155.420147,772.317060,0.0,0.0,11.633788,26.864989,601.983762,0.0,0.0,11.583597,85.978159,0.0,100.000000,7.118000
1,103.659208,-0.035119,34.352596,150.453257,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,7.220369,0.0,30.945258,1.517734
2,130.124424,-0.045282,130.619851,687.451257,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,33.912247,0.0,111.180000,3.605595
3,98.276855,-0.061393,330.643071,1610.936086,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,110.502614,0.0,255.000000,3.764000
4,98.326246,-0.074667,330.182418,1614.027909,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,111.570440,0.0,255.000000,3.589000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893,126.054181,9.873549,306.987833,1593.163663,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,87.037088,0.0,255.000000,1.108000
1894,125.527599,9.884029,300.933453,1568.599821,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,81.050055,0.0,255.000000,0.916000
1895,125.586853,9.889078,299.393638,1580.246944,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,79.429865,0.0,255.000000,1.204333
1896,125.948905,9.912293,306.290702,1570.158663,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,86.507599,0.0,255.000000,0.903667


In [6]:
df_capacity_0.to_csv('island_capacity_0.csv')
df_capacity_2020.to_csv('island_capacity_2020.csv')
df_capacity_2050.to_csv('island_capacity_2050.csv')